# Brewtiful: A Streamlit-Powered Assistant
This notebook contains the code for a Streamlit-based chatbot that interacts with markdown notes. The assistant is designed to help users manage and retrieve information from their notes efficiently. 

## Key Features

### LangChain
The application utilizes the **LangChain** library, a framework designed to simplify the development of applications powered by large language models (LLMs). LangChain provides modular components and tools that can be chained together to create sophisticated workflows. Its key benefits include:
- **Modularity**: Offers reusable components for common LLM tasks (e.g., interacting with models, managing prompts, retrieving data).
- **Composition**: Allows developers to easily combine these components into "chains" or more complex "agents" to build custom applications.
- **Integration**: Provides integrations with various LLM providers, data sources, and tools.


The application leverages NLP (natural language processing) techniques facilitated by LangChain to provide meaningful insights and answers to user queries:
- **Text Preprocessing**: Cleans and structures the raw text data for further analysis.
- **Embeddings**: Converts text into numerical vectors using models integrated via LangChain to capture semantic meaning, enabling similarity searches and document retrieval.
- **Text Splitting**: Breaks down large documents into smaller, manageable chunks using LangChain's text splitters to ensure efficient processing and context preservation.
- **Context-Aware Responses**: Uses LangChain's memory modules to maintain the flow of conversation, ensuring that responses are relevant to the user's query and the chat history.
- **Prompt Engineering**: Defines the behavior of the assistant through carefully crafted instructions using LangChain's prompt templates, ensuring accurate and helpful responses.

### Streamlit for User Interface
Streamlit is a powerful framework for building interactive web applications in Python. It allows developers to create user-friendly interfaces with minimal effort, making it an good choice for deploying machine learning and data science applications. In this project, Streamlit is used to create a simple yet effective interface for users to interact with the chatbot.

## Application Workflow Summary
The application follows these main steps, mirroring the structure of the code cells:

### 1. Set-Up
- 1.1.  **Import Required Libraries**: Load necessary Python packages from standard, third-party, and LangChain libraries.
- 1.2.  **Load Environment Variables**: Access the `GOOGLE_API_KEY` from a `.env` file.

### 2. Definition of Functions and Variables
- 2.1.  **Initialize Session State**: Define and run `initialize_session_state` to ensure variables persist across Streamlit reruns.
- 2.2.  **Define Data Loading and Preprocessing**: Create the `load_and_preprocess_notes` function to read markdown files, extract metadata, and handle errors.
- 2.3.  **Define Text Splitting**: Create the `split_documents` function using `RecursiveCharacterTextSplitter` to break notes into chunks.
- 2.4.  **Define Document Filtering**: Create the `filter_documents` function to allow selection based on metadata (title, categories, tags).
- 2.5.  **Define Document Formatting**: Create the `format_docs` function to prepare retrieved document content for display or input to the LLM.
- 2.6.  **Define Chat History Retrieval**: Create the `get_chat_history` function to extract conversation history from memory.
- 2.7.  **Define LangChain Chain Creation**: Create the `create_chain` function, which assembles the prompt template, retriever, memory access, and LLM into a runnable sequence.

### 3. Main Application Logic

- 3.1. **Define Main Application Logic (`main` function)**: This function orchestrates the application:
    - Sets the Streamlit page title and header.
    - Calls `initialize_session_state`.
    - Performs one-time loading/processing of notes, embeddings, vector store creation, LLM setup, and memory initialization, storing results in session state.
    - Sets up the Streamlit sidebar UI for filtering.
    - Sets up preloaded prompt buttons.
    - Displays the chat history.
    - Handles user input via `st.chat_input`.
    - If a query exists: displays it, filters documents, sets up the retriever, invokes the chain (if documents exist), handles the response (display, memory update), and resets button state.

- 3.2. **Run Application**: 
    - The `if __name__ == "__main__":` block calls the `main` function to start the Streamlit app when the script is executed directly.

---

## 1. Set-Up

### 1.1 Import Required Libraries

This section imports the necessary libraries. These include:
- **Standard libraries** like `os`, `pathlib`, and `yaml` for file, path management, and YAML parsing.
- **Third-party libraries** like `streamlit` for building the web app, and `dotenv` for loading environment variables.
- **LangChain components**: Specific modules from the LangChain framework are imported to handle core LLM-related tasks. This includes interfaces for chat models (`ChatGoogleGenerativeAI`), embedding models (`HuggingFaceEmbeddings`), vector stores (`FAISS`), prompt templating (`ChatPromptTemplate`), runnable components (`RunnableLambda`), document representation (`Document`), text splitting (`RecursiveCharacterTextSplitter`), and conversation memory (`ConversationBufferMemory`).
These libraries are essential for the functionality of the application, enabling tasks like file handling, environment configuration, and the complex NLP processing managed by LangChain.

In [ ]:
# --- Standard Library Imports ---
import os
from pathlib import Path
import yaml

# --- Third-Party Imports ---
import streamlit as st
from dotenv import load_dotenv

# --- LangChain imports ---
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory

### 1.2 Load Environment Variables
Environment variables are used to securely store sensitive information like API keys. This step loads these variables from a `.env` file using the `dotenv` library. For example, the `GOOGLE_API_KEY` is required to access Google's Generative AI services.

In [2]:
# --- Load Environment Variables ---
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

## 2. Definition of Functions and Variables

### 2.1 Initialize Session State
Streamlit uses a session state to persist data across user interactions. This function initializes variables to ensure they are available throughout the app. Without this step, the app would not be able to maintain state between user actions. 

The session state is a dictionary-like object (`st.session_state`) that stores data specific to a user's current browser session. It's crucial because Streamlit reruns the entire script on each interaction; session state allows variables (like chat history, loaded data, or model instances) to persist across these reruns, maintaining the application's context. As the user interacts with the chatbot (e.g., sending a message, applying filters), the relevant data (like the `messages` list or `user_query`) is updated within `st.session_state` by the application logic.

In [3]:
# --- Helper Functions ---
def initialize_session_state():
    """Initialize session state variables if they do not exist."""
    if "all_notes" not in st.session_state:
        st.session_state.all_notes = []
    if "documents" not in st.session_state:
        st.session_state.documents = []
    if "embeddings" not in st.session_state:
        st.session_state.embeddings = None
    if "db" not in st.session_state:
        st.session_state.db = None
    if "llm" not in st.session_state:
        st.session_state.llm = None
    if "memory" not in st.session_state:
        st.session_state.memory = None
    if "user_query" not in st.session_state:
        st.session_state.user_query = None
    if "messages" not in st.session_state:
        st.session_state.messages = []

### 2.2 Load and Preprocess Notes
Preprocessing involves reading markdown files, extracting their content and metadata, and preparing them for further analysis. The `yaml` library is used to parse the metadata (e.g., title, tags, categories) embedded in the markdown files' front matter. This step ensures that the notes are structured and ready for NLP tasks.

In [4]:
# --- Data Loading and Preprocessing ---
def load_and_preprocess_notes(notes_dir="Notes"):
    """
    Load and preprocess markdown notes from the specified directory.

    Args:
        notes_dir (str): Path to the directory containing markdown files.

    Returns:
        list: A list of tuples containing filename, content, and metadata.
    """
    all_notes = []
    notes_path = Path(notes_dir)

    if not notes_path.exists() or not notes_path.is_dir():
        st.error(f"Error: '{notes_dir}' is not a valid directory.")
        return []

    markdown_files = list(notes_path.rglob("*.md"))

    if not markdown_files:
        st.warning(f"No markdown files found in '{notes_dir}'.")
        return []

    for file_path in markdown_files:
        try:
            # Skip README.md
            if file_path.name.lower() == "readme.md":
                continue

            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()

            # Extract YAML front matter
            if content.startswith("---"):
                end_index = content.find("---", 3)
                if end_index != -1:
                    yaml_content = content[3:end_index].strip()
                    metadata = yaml.safe_load(yaml_content) if yaml_content else {}
                    content = content[end_index + 3:].strip()
                else:
                    metadata = {} # No closing '---' found
            else:
                metadata = {} # No front matter detected

            all_notes.append((file_path.name, content, metadata))
        except Exception as e:
            st.error(f"Error reading or processing {file_path}: {e}")
    return all_notes

### 2.3 Split Documents
Markdown notes are often lengthy, making it difficult to process them as a whole. This function splits the notes into smaller chunks using a text splitter. The `chunk_size` defines the maximum size of each chunk, while `chunk_overlap` ensures some overlap between chunks to preserve context. This step is crucial for NLP tasks like embeddings and retrieval.

In [5]:
# --- Text Splitting ---
def split_documents(notes_data, chunk_size=1000, chunk_overlap=50):
    """
    Split notes into smaller chunks for processing.

    Args:
        notes_data (list): List of tuples containing filename, content, and metadata.
        chunk_size (int): Maximum size of each chunk.
        chunk_overlap (int): Overlap between chunks.

    Returns:
        list: A list of Document objects.
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = []
    for filename, content, metadata in notes_data:
        split_texts = text_splitter.split_text(content)
        for text in split_texts:
            doc_metadata = {"source": filename, **metadata}
            docs.append(Document(page_content=text, metadata=doc_metadata))
    return docs

### 2.4 Filter Documents
Filtering allows users to narrow down the documents based on specific criteria like title, categories, or tags. This function iterates through the documents and applies the filters provided by the user. It checks if the selected categories/tags are a subset of the document's categories/tags. This step ensures that only relevant documents are retrieved for further processing.

In [6]:
# --- Filter Documents ---
def filter_documents(documents, selected_title=None, selected_categories=None, selected_tags=None):
    """
    Filter documents based on title, categories, and tags.

    Args:
        documents (list): List of Document objects.
        selected_title (str): Title to filter by.
        selected_categories (list): Categories to filter by.
        selected_tags (list): Tags to filter by.

    Returns:
        list: A list of filtered Document objects.
    """
    filtered_docs = documents

    if selected_title:
        filtered_docs = [doc for doc in filtered_docs if doc.metadata.get('title') == selected_title]

    if selected_categories:
        # Check if selected categories are a subset of document categories
        filtered_docs = [
            doc for doc in filtered_docs
            if doc.metadata.get('categories') is not None and set(selected_categories).issubset(set(doc.metadata.get('categories')))
        ]

    if selected_tags:
        # Check if selected tags are a subset of document tags
        filtered_docs = [
            doc for doc in filtered_docs
            if doc.metadata.get('tags') is not None and set(selected_tags).issubset(set(doc.metadata.get('tags')))
        ]

    return filtered_docs

### 2.5 Format Documents
Formatting prepares the documents for display in the Streamlit app. It creates a readable string representation of the document content and metadata, making it easier for users to understand the retrieved information.

In [7]:
# --- Format Docs ---
def format_docs(docs):
    """
    Format documents for display.

    Args:
        docs (list): List of Document objects.

    Returns:
        str: Formatted string representation of documents.
    """
    return "\n\n".join(f"Source: {doc.metadata['source']}\nContent: {doc.page_content}" for doc in docs)

### 2.6 Retrieve Chat History
The chat history is stored in memory to maintain context across user interactions. This function retrieves and formats the chat history, which is essential for generating context-aware responses.

**Note on memory duration**
In this application, the memory is stored as part of the Streamlit session state (`st.session_state`). This means:
- The memory persists only for the duration of the user's session in the browser.
- Once the session ends (e.g., the browser is closed or refreshed), the memory is cleared.
- The memory is not stored persistently across sessions or written to disk in this implementation. If persistent memory is required, additional mechanisms like database storage would need to be implemented.

In [8]:
# --- Chat History ---
def get_chat_history(memory):
    """
    Retrieve chat history from memory.

    Args:
        memory (ConversationBufferMemory): Memory object containing chat history.

    Returns:
        str: Formatted chat history.
    """
    messages = memory.chat_memory.messages
    return "\n".join([f"{m.type}: {m.content}" for m in messages])

### 2.7 Create LangChain Processing Chain
LangChain is a framework for building applications powered by language models. It allows developers to connect different components (like LLMs, data sources, and tools) into cohesive workflows called "Chains". This function creates a specific processing chain tailored for this application, combining a language model, a document retriever, and conversational memory.

#### Key LangChain Concepts Used Here:
- **LangChain Framework**: A library providing standard interfaces and integrations for building LLM applications. It simplifies tasks like prompt management, data connection, model interaction, and state management.
- **Chains**: The core concept in LangChain. Chains represent sequences of calls, either to LLMs or other utilities. They allow for structured workflows where the output of one step becomes the input for the next. This application uses a chain to structure the flow from receiving a user question to generating a response based on retrieved context and chat history.
- **Components**: LangChain provides various building blocks:
    - **LLMs/Chat Models**: Interfaces to interact with language models (e.g., `ChatGoogleGenerativeAI`).
    - **Prompts**: Templates for constructing the input to LLMs, often combining user input, context, and instructions (e.g., `ChatPromptTemplate`).
    - **Retrievers**: Components that fetch relevant data (like documents from `FAISS`) based on a query.
    - **Memory**: Mechanisms to store and recall information from previous interactions in a conversation (e.g., `ConversationBufferMemory`).
- **System Prompt**: A predefined instruction within the `ChatPromptTemplate` that sets the overall behavior and persona of the assistant. For example, the assistant is instructed to answer based on the provided context and chat history.
- **RunnableLambda**: A flexible component in LangChain's expression language (LCEL) that allows wrapping arbitrary functions into runnable parts of a chain. This enables dynamic data transformations at runtime, such as formatting documents (`format_docs`) or retrieving chat history (`get_chat_history`) within the chain's execution flow.

This chain integrates the retriever (for fetching relevant documents), the memory (for maintaining chat history), and the language model (for generating responses), orchestrated by the prompt template. Together, they form a cohesive pipeline for processing user queries contextually.

In [9]:
# --- Create Chain ---
def create_chain(llm, retriever, memory):
    """
    Create a LangChain processing chain.

    Args:
        llm (ChatGoogleGenerativeAI): Language model.
        retriever (FAISS): Document retriever.
        memory (ConversationBufferMemory): Memory object.

    Returns:
        dict: LangChain processing chain.
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant. Answer based on the context and the chat history. If you don't know, say so.\n\nContext:\n{context}\n\nChat History:\n{chat_history}"), # Removed "Answer only in rhymes."
        ("user", "{question}")
    ])

    return (
        {
            "context": retriever | RunnableLambda(format_docs),
            "question": RunnableLambda(lambda x: x),
            "chat_history": RunnableLambda(lambda x: get_chat_history(memory)),
        }
        | prompt
        | (llm if llm else RunnableLambda(lambda x: x)) # Pass through if llm is None
    )

## 3. Main Application Logic

### 3.1 Main Function Definition
The `main` function orchestrates the Streamlit application. It initializes the necessary components, handles user interactions, manages the application state, and integrates the LangChain elements to process queries and generate responses. Here's a breakdown of the key steps as they are initialized or used within this function:

1.  **Session State Initialization**: Ensures that variables like notes, documents, embeddings, vector store, LLM, and memory persist across user interactions within the same session using `st.session_state`. This is crucial for maintaining the context of the application between reruns triggered by user actions.
2.  **Data Loading and Processing**: Loads markdown notes from the specified directory, preprocesses them (extracting content and metadata), and splits them into manageable `Document` objects using `load_and_preprocess_notes` and `split_documents`. This step prepares the raw data for embedding and retrieval.
3.  **Embeddings Initialization**: If not already in the session state, numerical representations (embeddings) of text are generated using an embedding model (`HuggingFaceEmbeddings` with "BAAI/bge-small-en-v1.5"). These vectors capture semantic meaning, allowing for similarity comparisons between the query and the documents.
4.  **Vector Store Creation**: If not already in the session state, a `FAISS` vector store (`st.session_state.db`) is created using the generated embeddings and the processed documents. This store indexes the document vectors for efficient similarity searches.
5.  **LLM Initialization**: If not already in the session state, the language model (`ChatGoogleGenerativeAI` with "gemini-2.0-flash-lite") responsible for understanding the context and generating human-like responses is initialized, requiring the `GOOGLE_API_KEY`.
6.  **Memory Initialization**: If not already in the session state, conversational memory (`ConversationBufferMemory`) is set up to store the history of the chat interaction, allowing the LLM to generate contextually relevant responses based on previous turns.
7.  **User Interface (Sidebar Filters & Preloaded Prompts)**: Streamlit components are used to create the user interface.
    *   The sidebar dynamically populates filter options (title, categories, tags) based on the loaded documents and allows users to select criteria.
    *   Preloaded prompt buttons offer users quick ways to ask common questions, setting the `st.session_state.user_query`.
8.  **User Interaction (Chat Interface)**: The main chat interface displays the conversation history stored in `st.session_state.messages` and provides an input box (`st.chat_input`) for the user to ask questions, which also sets `st.session_state.user_query`.
9.  **Dynamic Filtering and Retrieval**: When `st.session_state.user_query` is set (either via input or button click), the application:
    *   Applies any active filters selected in the sidebar to the full document list using `filter_documents`; if no filters are selected, this step effectively uses the complete list of documents.
    *   If the resulting `filtered_docs` list is not empty, it creates a ***new, temporary***, filtered `FAISS` vector store (`db_filtered`) for this specific query using only these documents and the existing embeddings. A corresponding `dynamic_retriever` is then created from this temporary store. This ensures the subsequent search only considers documents matching the current filters. *(Note: Creating a new vector store on every query can be resource-intensive (CPU, memory, time) especially with many documents. This approach prioritizes filtering accuracy over performance; optimizations like caching the filtered store or post-retrieval filtering could be considered for larger scale applications.)*
    *   If `filtered_docs` is empty (due to filters or no documents loaded), it sets up a dummy retriever that returns nothing and issues the warning `"No documents match the filters."` using `st.warning()`.
10. **Chain Creation and Invocation**: If an LLM is available and there are documents to retrieve from (i.e., `filtered_docs` is not empty):
    *   The LangChain processing chain is created *dynamically* using `create_chain`, passing the LLM, the `dynamic_retriever` (which might be based on all documents or a filtered subset), and the `memory`.
    *   The chain is invoked with the `st.session_state.user_query`. This triggers the sequence: retrieve context -> get chat history -> format prompt -> call LLM.
11. **Response Handling**:
    *   The generated response content is extracted from the chain's output.
    *   Both the user query and the AI response are added to the `st.session_state.memory` and the `st.session_state.messages` list (for display).
    *   The response is displayed in the chat interface using `st.chat_message`.
    *   Error handling is included for potential issues during chain invocation.
    *   Specific handling is added for cases where filters result in no documents, providing a direct message to the user.
12. **State Reset**: If the query came from a button click, `st.session_state.user_query` is reset to `None` to prevent re-triggering on the next Streamlit rerun.



In [ ]:
# --- 3.1 Main Function Definition ---
def main():
    """Main function for the Streamlit app."""
    st.title("Welcome to Brewtiful ☕")
    st.subheader("What questions do you have for us?")

    # --- Step 1. Session State Initialization ---
    initialize_session_state()

    # --- Step 2. Data Loading and Processing ---
    if not st.session_state.all_notes:
        st.session_state.all_notes = load_and_preprocess_notes()
        st.session_state.documents = split_documents(st.session_state.all_notes) if st.session_state.all_notes else []

    # --- LangChain Component Setup (Steps 3-6) ---
    if st.session_state.documents:
    # --- Step 3. Embeddings Initialization ---
        if not st.session_state.embeddings:
            st.session_state.embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
    # --- Step 4. Vector Store Creation ---
        if not st.session_state.db:
            st.session_state.db = FAISS.from_documents(st.session_state.documents, st.session_state.embeddings)
    # --- Step 5. LLM Initialization ---
        if not st.session_state.llm and GOOGLE_API_KEY:
            # Updated model name
            st.session_state.llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", google_api_key=GOOGLE_API_KEY, temperature=0.5)
        elif not st.session_state.llm:
            # Set LLM to None if key is missing, chain will handle this
            st.session_state.llm = None
    # --- Step 6. Memory Initialization ---
        if not st.session_state.memory:
            st.session_state.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, input_key="question")
    else:
        st.warning("No documents available.")
        return # Exit if no documents to process

    # --- Step 7. User Interface Setup(Sidebar Filters & Preloaded Prompts) ---
    with st.sidebar:
        st.header("Filters")

        unique_titles = []
        unique_categories = []
        unique_tags = []

        # Populate filter options from document metadata
        for doc in st.session_state.documents:
            title = doc.metadata.get('title')
            if title is not None:
                unique_titles.append(str(title))
            categories = doc.metadata.get('categories')
            if categories:
                unique_categories.extend(categories)
            tags = doc.metadata.get('tags')
            if tags:
                unique_tags.extend(tags)

        # Get unique sorted lists for dropdowns/multiselects
        unique_titles = sorted(list(set(unique_titles)))
        unique_categories = sorted(list(set(unique_categories)))
        unique_tags = sorted(list(set(unique_tags)))

        # Display filter widgets
        selected_title = st.selectbox("Select Title", options=[None] + unique_titles)
        selected_categories = st.multiselect("Select Categories", options=unique_categories)
        selected_tags = st.multiselect("Select Tags", options=unique_tags)

    # Preloaded Prompts (Buttons)
    st.write("Quick Questions:")
    col1, col2, col3 = st.columns(3)

    # Set user_query in session state if a button is clicked
    if col1.button("Can you provide specific details about brewtiful?", key="button1"):
        st.session_state.user_query = "Can you provide specific details about brewtiful?"
    if col2.button("What is the current forecast for Enterprise clients?", key="button2"):
        st.session_state.user_query = "What is the current forecast for Enterprise clients?"
    if col3.button("Describe the ongoing activities related to the cloud.", key="button3"):
        st.session_state.user_query = "Describe the ongoing activities related to the cloud."

    # --- Step 8. User Interaction (Chat Interface) ---
    # Display existing messages
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # Get new user input
    user_query_input = st.chat_input("Ask a question...")
    if user_query_input:
        st.session_state.user_query = user_query_input

    # --- Query Handling (Steps 9-12) ---
    if st.session_state.user_query:
        # Display user query
        with st.chat_message("user"):
            st.markdown(st.session_state.user_query)
        # Add user query to message history for display (Part of Step 11)
        st.session_state.messages.append({"role": "user", "content": st.session_state.user_query})

    # --- Step 9. Dynamic Filtering and Retrieval Setup ---
        filtered_docs = filter_documents(st.session_state.documents, selected_title, selected_categories, selected_tags)

        if filtered_docs:
            # Create a temporary filtered vector store and retriever if docs remain after filtering
            db_filtered = FAISS.from_documents(filtered_docs, st.session_state.embeddings)
            dynamic_retriever = db_filtered.as_retriever(search_kwargs={"k": 3})
        else:
            # If filtering removes all docs, create a dummy retriever and warn
            dynamic_retriever = RunnableLambda(lambda x: []) # Returns an empty list
            st.warning("No documents match the filters.") # Warning displayed in main area

    # --- Step 10. Chain Creation and Invocation ---
        # Proceed only if documents were found after filtering
        if filtered_docs:
            chain = create_chain(st.session_state.llm, dynamic_retriever, st.session_state.memory)
            try:
                with st.spinner("Thinking..."):
                    # Get response object
                    response_obj = chain.invoke(st.session_state.user_query)
                # Extract the content from the response object
                response_content = response_obj.content if hasattr(response_obj, 'content') else str(response_obj)

    # --- Step 11. Response Handling (Success) ---
                # Add interaction to memory
                st.session_state.memory.chat_memory.add_user_message(st.session_state.user_query)
                st.session_state.memory.chat_memory.add_ai_message(response_content)

                # Display assistant response
                with st.chat_message("assistant"):
                    st.markdown(response_content)
                # Add assistant response to message history for display
                st.session_state.messages.append({"role": "assistant", "content": response_content})

    # --- Step 12. State Reset ---
                # Clear button-pressed query state only if it wasn't from chat_input this run
                if st.session_state.user_query != user_query_input:
                    st.session_state.user_query = None

            except Exception as e:
                # Catch-all for errors during chain creation or invocation
                st.error(f"An error occurred: {e}")
        


### 3.2 Run Application
The `if __name__ == "__main__":` block ensures that the `main()` function is called only when the script is executed directly (not when imported as a module). This is the standard Python entry point for running the application.

In [ ]:
# --- 3.2 Run Application ---
if __name__ == "__main__":
    main()